# Question 4: Secure, Non-Executable Weight Loading



## Problem Statement



> Assume you have a network’s weights that need to be loaded. Write a code snippet for loading these weights **without using Hugging Face SafeTensors or similar serialization formats**. The method must ensure the model’s weights can be loaded **without executing malicious code in the background**. Justify your approach with a description of the use case and the security mechanism employed.



This notebook presents a **simple, self-contained solution** based on:



- A safe array container format (`.npz` from NumPy, which does not execute code)

- A manifest describing expected parameter names, shapes, dtypes, and checksums

- A cryptographic signature over the weight file (Ed25519)

- A strict PyTorch loader that validates everything before loading into the model



No use is made of `safetensors`, `pickle`, or `torch.load` on untrusted data. Instead, we treat the weight file as opaque bytes, verify it cryptographically, and parse it only as plain numeric arrays.

## Use Case and Security Goals


### Use Case



You receive a file (for example, `weights.npz`) from a third party that contains the parameters for a neural network. You need to load these weights into your own PyTorch model instance, but you **do not fully trust the file source** and must ensure that loading it **cannot execute hidden code**.



### Security Risk (Why not `pickle` / `torch.load` from untrusted sources?)



- Formats such as `torch.save` / `torch.load` rely on Python’s `pickle` under the hood.

- Pickle can reconstruct arbitrary Python objects, which may define special methods such as `__reduce__` or `__setstate__` that execute code during deserialization.

- A malicious weight file could, for example, run `os.system("echo hello world")` (or worse) **simply by being loaded**.



### Security Goals of the Proposed Method



1. **No code execution on load**  

   Use only data formats and APIs that do not execute Python objects (NumPy `.npz` with `allow_pickle=False`).



2. **Integrity and authenticity**  

   Ensure that the file is exactly what a trusted producer created, using a detached cryptographic signature (Ed25519) verified before parsing the file.



3. **Structural validation**  

   Check that parameter names, shapes, dtypes, and per-array SHA-256 checksums match an expected manifest before loading anything into the model.



4. **Strict model loading**  

   Use `model.load_state_dict(..., strict=True)` so that any mismatch in parameter keys or shapes raises an error rather than being silently ignored.

## Rationale for the Chosen Format (`.npz`)



- **No executable objects**: NumPy's `.npz` format stores raw arrays only; when opened with `allow_pickle=False`, it does not deserialize arbitrary Python objects and therefore cannot run code.



- **Simple and widely supported**: `.npz` is a standard container for NumPy arrays and is easy to inspect.



- **Separation of concerns**: We add integrity and authenticity through a separate cryptographic signature and a JSON manifest, not via the serialization format itself.





This directly addresses the assignment's requirement: we avoid SafeTensors and similar specialized formats, and instead use a simple array container plus explicit, clearly described security mechanisms.


## Secure Loader: Core Code Snippet (Consumer Side)



The following function shows how a consumer (the party loading the weights) can securely load network weights from a `.npz` file **without executing any code contained in the file**.



Key security properties:



- Verifies a **detached Ed25519 signature** over the entire `.npz` file before any parsing occurs.

- Loads arrays with `np.load(..., allow_pickle=False)` to prevent any pickled objects from being deserialized or executed.

- Uses a **manifest** to enforce expected parameter names, shapes, dtypes, and per-array SHA-256 checksums.

- Uses `model.load_state_dict(..., strict=True)` to reject unexpected or missing parameters.



This directly prevents even a simple hidden payload such as a “hello world” command from being executed, because the loader never unpickles arbitrary Python objects and treats the file strictly as verified numeric data.

In [1]:
"""
Secure loader for model weights stored in a .npz file.

Prerequisites (consumer side):
    pip install cryptography numpy torch

Assumes the following files are provided by a trusted producer:
    - weights.npz        : NumPy .npz file with arrays for each parameter name
    - weights.npz.sig    : detached Ed25519 signature over weights.npz
    - public_key.pem     : producer's Ed25519 public key in PEM format
    - manifest.json      : JSON manifest with expected shapes/dtypes/checksums

This loader:
    - Verifies the signature before parsing the .npz
    - Loads arrays with allow_pickle=False (no code execution)
    - Checks names, shapes, dtypes and per-array SHA-256 against the manifest
    - Loads the verified tensors into a PyTorch model with strict=True
"""

import io
import json
import hashlib
import numpy as np
import torch

from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature


def sha256_bytes(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()


def verify_signature_with_pem(public_key_pem_path: str, message: bytes, signature: bytes) -> None:
    """Verify Ed25519 signature; raise ValueError if verification fails."""
    with open(public_key_pem_path, "rb") as f:
        pub_pem = f.read()
    public_key = serialization.load_pem_public_key(pub_pem)

    try:
        public_key.verify(signature, message)
    except InvalidSignature as exc:
        raise ValueError("Signature verification failed: untrusted or tampered file.") from exc


def load_weights_npz_secure(
    model: torch.nn.Module,
    weights_file: str,
    signature_file: str,
    public_key_pem: str,
    manifest_file: str,
) -> None:
    """Securely load weights into a PyTorch model from a signed .npz file.

    Parameters
    ----------
    model : torch.nn.Module
        The model instance whose parameters will be loaded.
    weights_file : str
        Path to the .npz file produced by the trusted party.
    signature_file : str
        Path to the detached Ed25519 signature for weights_file.
    public_key_pem : str
        Path to the producer's public key in PEM format.
    manifest_file : str
        Path to JSON manifest describing parameters: names, shapes, dtypes, sha256.
    """

    # 1) Read raw weight bytes and verify signature BEFORE parsing.
    with open(weights_file, "rb") as f:
        raw_bytes = f.read()
    with open(signature_file, "rb") as f:
        signature = f.read()

    verify_signature_with_pem(public_key_pem, raw_bytes, signature)

    # 2) Load manifest (expected param names/shapes/dtypes/checksums)
    with open(manifest_file, "r", encoding="utf-8") as f:
        manifest = json.load(f)

    # Expected manifest format:
    # {"params": {
    #     "layer1.weight": {"shape": [64, 128], "dtype": "float32", "sha256": "..."},
    #     "layer1.bias":   {"shape": [64],      "dtype": "float32", "sha256": "..."}
    # }}

    expected_params = manifest.get("params", {})
    expected_keys = set(expected_params.keys())

    # 3) Load npz from memory with allow_pickle=False to avoid executing objects.
    npz = np.load(io.BytesIO(raw_bytes), allow_pickle=False)

    artifact_keys = set(npz.files)
    if artifact_keys != expected_keys:
        raise ValueError(
            "Artifact parameter keys do not match manifest. "
            f"Artifact keys: {sorted(artifact_keys)}, Manifest keys: {sorted(expected_keys)}"
        )

    # 4) Validate each array and build state_dict for the model.
    state_dict = {}
    model_keys = set(model.state_dict().keys())

    for name in npz.files:
        if name not in model_keys:
            raise ValueError(f"Param '{name}' is not expected by the model (potential mismatch).")

        arr = npz[name]
        meta = expected_params[name]

        # Shape check
        if list(arr.shape) != meta["shape"]:
            raise ValueError(
                f"Shape mismatch for '{name}': artifact {list(arr.shape)} != manifest {meta['shape']}"
            )

        # Dtype check
        if str(arr.dtype) != meta["dtype"]:
            raise ValueError(
                f"dtype mismatch for '{name}': artifact {arr.dtype} != manifest {meta['dtype']}"
            )

        # Per-array checksum check
        arr_sha = sha256_bytes(arr.tobytes())
        if arr_sha != meta["sha256"]:
            raise ValueError(
                f"Checksum mismatch for '{name}': artifact {arr_sha} != manifest {meta['sha256']}"
            )

        # Convert to torch tensor; copy() avoids npz mmap issues
        tensor = torch.from_numpy(arr.copy())
        state_dict[name] = tensor

    # 5) Strictly load into model; this will fail on any remaining mismatch.
    model.load_state_dict(state_dict, strict=True)
    print("Weights loaded successfully and signature + manifest validated.")


## End-to-End Example (Producer and Consumer)



The following self-contained script demonstrates the **entire workflow** in one place:



1. **Producer side** (trusted machine):

   - Defines a small PyTorch model.

   - Exports its parameters to `weights.npz` (NumPy container, data only).

   - Builds `manifest.json` with shapes, dtypes, and SHA-256 checksums for each parameter.

   - Generates an Ed25519 key pair.

   - Signs `weights.npz` to produce `weights.npz.sig`.



2. **Consumer side** (your environment, untrusted file):

   - Loads the public key from `public_key.pem`.

   - Verifies the Ed25519 signature on `weights.npz`.

   - Loads arrays with `np.load(..., allow_pickle=False)` (no code execution).

   - Validates all metadata (names, shapes, dtypes, checksums) against the manifest.

   - Safely imports tensors into a fresh model instance using `load_state_dict(strict=True)`.



This directly answers the assignment question: **weights are loaded without using SafeTensors or similar formats, and without any possibility of hidden code execution**, due to the combination of a non-executable data format, cryptographic signature verification, and strict structural validation.

In [2]:
"""
Full end-to-end example: producer + consumer using a secure .npz + signature pipeline.

Run this script top-to-bottom to see the complete flow.
"""

import torch
import torch.nn as nn
import numpy as np
import json
import hashlib
import io

from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature


print("\n=== PRODUCER SIDE (trusted) ===")


# 1. Define a tiny model
class TinyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        return self.fc2(torch.relu(self.fc1(x)))


producer_model = TinyModel()

# Extract state_dict to plain NumPy arrays
arrays = {}
for name, tensor in producer_model.state_dict().items():
    arrays[name] = tensor.cpu().numpy().astype("float32")


# 2. Save weights.npz (non-executable container)
np.savez("weights.npz", **arrays)
print("Saved weights.npz")


# 3. Create manifest.json with shape/dtype/sha256
manifest = {"params": {}}
for name, arr in arrays.items():
    sha = hashlib.sha256(arr.tobytes()).hexdigest()
    manifest["params"][name] = {
        "shape": list(arr.shape),
        "dtype": str(arr.dtype),
        "sha256": sha,
    }

with open("manifest.json", "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

print("Created manifest.json")


# 4. Generate Ed25519 signing key pair
private_key = Ed25519PrivateKey.generate()
public_key = private_key.public_key()

# Save public key as PEM (to distribute to consumers)
public_bytes = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo,
)
with open("public_key.pem", "wb") as f:
    f.write(public_bytes)

# 5. Sign weights.npz
raw_bytes = open("weights.npz", "rb").read()
signature = private_key.sign(raw_bytes)
with open("weights.npz.sig", "wb") as f:
    f.write(signature)

print("Generated weights.npz.sig")
print("Producer side completed.")


print("\n=== CONSUMER SIDE (untrusted file loader) ===")


# New model instance (simulating a fresh consumer environment)
consumer_model = TinyModel()


def sha256_bytes(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()


def verify_signature(public_key_pem_path: str, message: bytes, signature: bytes) -> None:
    with open(public_key_pem_path, "rb") as f:
        pub_bytes = f.read()
    pub = serialization.load_pem_public_key(pub_bytes)

    try:
        pub.verify(signature, message)
    except InvalidSignature as exc:
        raise ValueError("ERROR: Signature verification failed (file tampered).") from exc


def load_weights_npz_secure_demo(
    model: nn.Module,
    weights_file: str,
    signature_file: str,
    public_key_pem: str,
    manifest_file: str,
) -> None:
    """Simplified secure loader used in this end-to-end demo."""

    # Read raw bytes and signature
    raw = open(weights_file, "rb").read()
    sig = open(signature_file, "rb").read()

    # 1) Verify signature BEFORE any parsing
    verify_signature(public_key_pem, raw, sig)

    # 2) Load manifest
    with open(manifest_file, "r", encoding="utf-8") as f:
        manifest = json.load(f)

    # 3) Load npz safely (no pickling)
    npz = np.load(io.BytesIO(raw), allow_pickle=False)

    state_dict = {}
    model_keys = set(model.state_dict().keys())

    for key in npz.files:
        if key not in model_keys:
            raise ValueError(f"Unexpected parameter '{key}' in artifact.")

        arr = npz[key]
        meta = manifest["params"][key]

        # Shape validation
        if list(arr.shape) != meta["shape"]:
            raise ValueError(f"Shape mismatch for {key}")

        # Dtype validation
        if str(arr.dtype) != meta["dtype"]:
            raise ValueError(f"dtype mismatch for {key}")

        # Checksum validation
        if sha256_bytes(arr.tobytes()) != meta["sha256"]:
            raise ValueError(f"Checksum mismatch for {key}")

        state_dict[key] = torch.from_numpy(arr.copy())

    # 4) Strict loading into the model
    model.load_state_dict(state_dict, strict=True)
    print("Secure load successful: signature + manifest validated.")


# Run the secure loader on the consumer side
load_weights_npz_secure_demo(
    consumer_model,
    "weights.npz",
    "weights.npz.sig",
    "public_key.pem",
    "manifest.json",
)

print("\nAll checks passed. Model parameters imported securely.")



=== PRODUCER SIDE (trusted) ===
Saved weights.npz
Created manifest.json
Generated weights.npz.sig
Producer side completed.

=== CONSUMER SIDE (untrusted file loader) ===
Secure load successful: signature + manifest validated.

All checks passed. Model parameters imported securely.


## Overall Summary and Connection to the Question



### What the code does



This notebook proposes and implements a secure procedure for loading neural network weights received from an untrusted source, **without** using Hugging Face SafeTensors or similar serialization formats, and **without** executing malicious code in the background.



The approach is based on four main steps:



1. **Non-executable storage**  

   We store weights in a NumPy `.npz` file, which contains only raw arrays. When opened with `allow_pickle=False`, it does not deserialize arbitrary Python objects and therefore cannot run code.



2. **Cryptographic authenticity**  

   The producer signs the `weights.npz` file with an Ed25519 private key, and the consumer verifies this signature using the corresponding public key before parsing the file.



3. **Manifest-based structural checks**  

   A `manifest.json` file specifies, for each parameter, the expected name, shape, dtype, and SHA-256 checksum. The loader checks that every array in the file matches this manifest.



4. **Strict model loading in PyTorch**  

   After all checks pass, the validated arrays are converted to tensors and loaded into the model with `model.load_state_dict(..., strict=True)`, which rejects missing or extra parameters.



### How this answers the assignment prompt



- **No SafeTensors or similar formats**: We use plain NumPy `.npz` files plus a JSON manifest and a detached signature, rather than any specialized weight format.



- **No background code execution**: The loader never calls `pickle.loads`, `torch.load` on untrusted data, or any API that can construct arbitrary Python objects. As a result, even a trivial payload that tries to print "hello world" or call `os.system` cannot run.



- **Clear security mechanism**: The security relies on (i) avoiding executable serialization mechanisms, (ii) verifying integrity and authenticity with public-key signatures, and (iii) enforcing a strict, manifest-based structure before loading weights into the model.



Together, these elements meet the requirement to load model weights from an untrusted source safely, while providing a clear and justifiable security mechanism.

## Brief Justification (Exam-Oriented Summary)



- **Use case**: Loading neural network weights obtained from an external or untrusted party into a local PyTorch model.



- **Constraint**: The solution must not rely on Hugging Face SafeTensors or similar specialized serialization formats, and must ensure that loading the weights cannot execute hidden code (even something as simple as a “hello world” command).



### Why this approach is secure



1. **Safe container format (`.npz`)**

   - `.npz` stores only numeric arrays and simple metadata.

   - Using `np.load(..., allow_pickle=False)` prevents any pickled Python objects from being deserialized, eliminating code execution via object reconstruction.



2. **Cryptographic signature (Ed25519)**

   - A detached signature is created over the entire `weights.npz` file.

   - The consumer verifies this signature using the producer’s public key before interpreting the file. Any modification to the file causes verification to fail, and loading is aborted.



3. **Manifest-based validation**

   - A JSON manifest lists each expected parameter with its name, shape, dtype, and SHA-256 checksum.

   - The loader checks that every array in the file matches this specification before it is accepted.



4. **Strict loading into the model**

   - `model.load_state_dict(..., strict=True)` ensures that only the exact expected parameters are loaded, and that no unexpected or missing parameters are silently ignored.



### Why this prevents hidden code execution



- No untrusted code is ever deserialized: the loader treats the file as raw bytes, verifies it cryptographically, and then parses it only as plain arrays with `allow_pickle=False`.

- There is no use of `pickle`, `torch.load` on untrusted input, or any mechanism that could invoke `__reduce__` or other special methods.

- As a result, even a deliberately constructed malicious file cannot cause hidden code execution during the weight-loading process.



This provides a concise, technically sound justification that directly matches the assignment’s requirements.